<a href="https://colab.research.google.com/github/ulugeyik/toucanpi/blob/main/toucanpiISSdata1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#26/05/2022 Received AstroPi data from ISS. 
#This notebook is for carrying out general analysis of the data, getting an overview
# ToucanPi Team. Planned by Turgut Durduran

In [ ]:
#First step is to figure out how to get the data into "Google Colaboratory" notebook which is a variation of Jupyter Notebook for Python.

In [2]:
#the following part is needed in order to upload a data file.
#it requires that I turn on third part cookies
#solution: https://github.com/googlecolab/colabtools/issues/17
#once it has been uploaded I can comment it out.
#info: https://medium.com/towards-data-science/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92
#from google.colab import files
#uploaded = files.upload()

#for loading data on google collaboratory
#import pandas as pd
#import io
#mydata = pd.read_csv(io.BytesIO(uploaded['toucandata_20220513_201103.csv']),parse_dates={ 'newdate': ['Date/Time'] })

#this options is annoying because it seems to require that I keep uploading for every sessions.

# Code to read csv file into Colaboratory:
#the above link seems outdated, updated one is
#https://www.geeksforgeeks.org/ways-to-import-csv-files-in-google-colab/
#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link='https://drive.google.com/file/d/1P5RZMVeQMj-4LJr1XM0yQOQ6e00SkvzM/view?usp=sharing'
 
# to get the id part of the file
id = link.split("/")[-2]
print (id) # Verify that you have everything after 'd/' 
downloaded = drive.CreateFile({'id':id})
myfilename = 'toucandata_20220513_201103.csv'
downloaded.GetContentFile(myfilename) 


1P5RZMVeQMj-4LJr1XM0yQOQ6e00SkvzM


In [3]:
#for loading data on google collaboratory
import pandas as pd
mydata = pd.read_csv(myfilename,parse_dates={ 'newdate': ['Date/Time'] })


In [ ]:
# This method allows us to load data to the notebook.
#####

# Next step is to do some exploratory analysis of the data.

**Step 1:**  Photos


*   406 photos were sent.
*   To take a quick look we did an animated gif using linux tools: 
   " ffmpeg  -framerate 10 -pattern_type glob -i '*.jpg' -r 15 -vf scale=512:-1 out.gif "
* Initial impressions: (1) land not visible in most photos, (2) perhaps we took photos too infrequently.
* Email received: 
Good afternoon

We thought you might like to know that during team Toucan Pi's experiment, Canadarm 2 crossed the window of the ISS. ESA have let us know that this occurred on 73 of 406 images. 

I hope this does not affect your analysis. You are one of only two teams that captured this during the challenge this year! 

best

Sam
Astro Pi Team
* We might not be able to calculate velocity from these photos but it is not important.



**Step 2**: General look at all the data prior to interpretation.

In [4]:
#for maths,plots and other things
# First used pyplot which is great but less interactive on the online notebook
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
#if using plotly, this is more interactive.
import plotly.express as px
import plotly.graph_objs as go
#general
import numpy as np
import math

In [5]:


# Just kept for Reference on how to use pyplot
#plot acceleration, acceleration intensity of the axis in Gs.
#this is using pyplot , matplotlib, limited interaction capabilities, specially in Junyper/Collaboratory
#plt.figure()
#plt.plot(mydata.newdate,np.sqrt(np.square(mydata.AccX)+np.square(mydata.AccY)+np.square(mydata.AccZ)))
#locator = mdates.AutoDateLocator()
#formatter = mdates.ConciseDateFormatter(locator)
#plt.gca().xaxis.set_major_locator(locator)
#plt.gca().xaxis.set_major_formatter(formatter)
#plt.show()


In [6]:
#one option kept for reference
# #plot acceleration, acceleration intensity of the axis in Gs.
# #this is using plotly, increased interaction capabilities, specially in Junyper/Collaboratory
# # examples: https://plotly.com/python/line-and-scatter/
# #labels: https://plotly.com/python/figure-labels/
# #hover stuff https://plotly.com/python/hover-text-and-formatting/
# #note: plotly express, pxline etc is more capable in terms of hover stuff but you cannot add individual objects
# #updates: https://plotly.com/python/creating-and-updating-figures/

# fig= go.Figure(px.line(x=mydata.newdate,y=np.sqrt(np.square(mydata.AccX)+np.square(mydata.AccY)+np.square(mydata.AccZ)),hover_data=[mydata.Lat,mydata.Long],markers=True,labels={
#                      "x": "Date/time",
#                      "y": "Acceleration [G]",
#                      "hover_data_0": "Latitude (o)",
#                      "hover_data_1": "Longitude (o)"
#                                       }))
# fig.add_trace(go.Scatter(x=mydata.newdate,y=mydata.AccX,mode='lines+markers',name="Acceleration X"))
# fig.add_trace(go.Scatter(x=mydata.newdate,y=mydata.AccY,mode='lines+markers',name="Acceleration Y"))
# fig.add_trace(go.Scatter(x=mydata.newdate,y=mydata.AccZ,mode='lines+markers',name="Acceleration Z"))
# fig.update_layout(title='Acceleration over time',showlegend=True,hovermode='x unified')
# fig.show()



In [7]:
#Hack for labels: https://stackoverflow.com/questions/64371174/plotly-how-to-change-variable-label-names-for-the-legend-in-a-plotly-express-li
newnames = {'AccX':'Acceleration X', 'AccY': 'Acceleration Y', 'AccZ': 'Acceleration Z','wide_variable_3': 'Acceleration Total','Lat': 'Latitude', 'Long': 'Longitude'}
fig= go.Figure(px.line(mydata,x='newdate',y=['AccX','AccY','AccZ',np.sqrt(np.square(mydata.AccX)+np.square(mydata.AccY)+np.square(mydata.AccZ))],hover_data=['Lat','Long'],markers=True))
fig.update_layout(title='Acceleration over time',showlegend=True,yaxis_title="Acceleration [G]",legend_title="Data type",xaxis_title="Date/Time",hovermode='x')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()

In [8]:
newnames = {'MagnetX':'Magnetic F. X', 'MagnetY':'Magnetic F. Y','MagnetZ':'Magnetic F. Z','wide_variable_3': 'Magnetic F Total','Lat': 'Latitude', 'Long': 'Longitude'}
fig= go.Figure(px.line(mydata,x='newdate',y=['MagnetX','MagnetY','MagnetZ',np.sqrt(np.square(mydata.MagnetX)+np.square(mydata.MagnetY)+np.square(mydata.MagnetZ))],hover_data=['Lat','Long'],markers=True))
fig.update_layout(title='Magnetic Field over time',showlegend=True,yaxis_title="$\\text{Magnetic Field} [\mu T]$",legend_title="Data type",xaxis_title="Date/Time",hovermode='x')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()


In [9]:
#is there a latitude/longitude dependence here?
newnames = {'MagnetX':'Magnetic F. X', 'MagnetY':'Magnetic F. Y','MagnetZ':'Magnetic F. Z','wide_variable_3': 'Magnetic F Total','Lat': 'Latitude', 'Long': 'Longitude'}
fig= go.Figure(px.line(mydata,x='newdate',y=['MagnetX','MagnetY','MagnetZ',np.sqrt(np.square(mydata.MagnetX)+np.square(mydata.MagnetY)+np.square(mydata.MagnetZ)),'Lat','Long'],markers=True))
fig.update_layout(title='Magnetic Field & Position over time',showlegend=True,yaxis_title="$\\text{Magnetic Field} [\mu T]$",legend_title="Data type",xaxis_title="Date/Time",hovermode='x')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()

In [16]:
#show location on map with yaw, possibly wrong:
fig = px.scatter_geo(mydata,lat=mydata.Lat,lon=mydata.Long,color=mydata.MagnetZ,hover_data=[mydata.newdate])
fig.update_layout(title = 'ISS trajectory over world map & total magnetic field', title_x=0.5,coloraxis_colorbar=dict(title="Magnetic Field [uT]"))
fig.show()

In [11]:

#plot gyro data, radians/second ("rotational intensity")
newnames = {'GyroX':'Gyro. X', 'GyroY':'Gyro. Y','GyroZ':'Gyro. Z','wide_variable_3': 'Gyro. Total','GyroTotal': 'Gyro. Total','Lat': 'Latitude', 'Long': 'Longitude'}
tmp={'newdate': mydata.newdate, 'GyroX': mydata.GyroX*180.0 / math.pi, 'GyroY': mydata.GyroY*180.0 / math.pi,'GyroZ': mydata.GyroZ*180.0 / math.pi, 'GyroTotal' : np.sqrt(np.square(mydata.GyroX)+np.square(mydata.GyroY)+np.square(mydata.GyroZ)),'Lat': mydata.Lat, 'Long': mydata.Long}
tmpdf = pd.DataFrame(data=tmp)
fig= go.Figure(px.line(tmpdf,x='newdate',y=['GyroX','GyroY','GyroZ','GyroTotal'],hover_data=['Lat','Long'],markers=True))
fig.update_layout(title='Gyroscope data over time',showlegend=True,yaxis_title="$\\text{Gyroscope} [^o/s]$",legend_title="Data type",xaxis_title="Date/Time",hovermode='x')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()



In [12]:
#plot orientation data
tmp={'newdate': mydata.newdate, 'OrientPitch': mydata.OrientPitch*180.0 / math.pi, 'OrientRoll': mydata.OrientRoll*180.0 / math.pi,'OrientYaw': mydata.OrientYaw*180.0 / math.pi, 'Lat': mydata.Lat, 'Long': mydata.Long}
tmpdf = pd.DataFrame(data=tmp)
newnames = {'OrientPitch':'Pitch', 'OrientRoll':'Roll','OrientYaw':'Yaw','Lat': 'Latitude', 'Long': 'Longitude'}
fig= go.Figure(px.line(tmpdf,x='newdate',y=['OrientPitch','OrientRoll','OrientYaw'],hover_data=['Lat','Long'],markers=True))
fig.update_layout(title='Orientation over time',showlegend=True,yaxis_title="Orientation [$^o$]",legend_title="Data type",xaxis_title="Date/Time",hovermode='x')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()


# plt.figure()
# plt.plot(mydata.newdate,mydata.OrientPitch*180.0 / math.pi)
# locator = mdates.AutoDateLocator()
# formatter = mdates.ConciseDateFormatter(locator)
# plt.gca().xaxis.set_major_locator(locator)
# plt.gca().xaxis.set_major_formatter(formatter)
# plt.show()


# #plot orientation data
# plt.figure()
# plt.plot(mydata.newdate,mydata.OrientRoll*180.0 / math.pi)
# locator = mdates.AutoDateLocator()
# formatter = mdates.ConciseDateFormatter(locator)
# plt.gca().xaxis.set_major_locator(locator)
# plt.gca().xaxis.set_major_formatter(formatter)
# plt.show()



# #plot orientation data
# plt.figure()
# plt.plot(mydata.newdate,mydata.OrientYaw*180.0 / math.pi)
# locator = mdates.AutoDateLocator()
# formatter = mdates.ConciseDateFormatter(locator)
# plt.gca().xaxis.set_major_locator(locator)
# plt.gca().xaxis.set_major_formatter(formatter)
# plt.show()



In [13]:

#yaw contains the most interesting information.
tmp={'newdate': mydata.newdate, 'OrientPitch': mydata.OrientPitch*180.0 / math.pi, 'OrientRoll': mydata.OrientRoll*180.0 / math.pi,'OrientYaw': mydata.OrientYaw*180.0 / math.pi, 'Lat': mydata.Lat, 'Long': mydata.Long}
tmpdf = pd.DataFrame(data=tmp)
newnames = {'OrientPitch':'Pitch', 'OrientRoll':'Roll','OrientYaw':'Yaw','Lat': 'Latitude', 'Long': 'Longitude'}
fig= go.Figure(px.line(tmpdf,x='newdate',y=['OrientYaw','Lat','Long'],markers=True))
fig.update_layout(title='Orientation & Position over time',showlegend=True,yaxis_title="[$^o$]",legend_title="Data type",xaxis_title="Date/Time",hovermode='x')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()

#plot orientation data
# plt.figure()
# plt.plot(mydata.newdate,mydata.OrientYaw*180.0 / math.pi, label='Yaw')
# plt.plot(mydata.newdate,mydata.Long,label='Longitude')
# plt.plot(mydata.newdate,mydata.Lat,label='Latitude')
# locator = mdates.AutoDateLocator()
# formatter = mdates.ConciseDateFormatter(locator)
# plt.gca().xaxis.set_major_locator(locator)
# plt.gca().xaxis.set_major_formatter(formatter)
# plt.gca().legend()
# plt.show()


In [14]:
#show location on map with yaw, possibly wrong:
fig = px.scatter_geo(mydata,lat=mydata.Lat,lon=mydata.Long,color=mydata.OrientYaw*180/math.pi,hover_data=[mydata.newdate])
fig.update_layout(title = 'ISS trajectory over world map & Yaw', title_x=0.5,coloraxis_colorbar=dict(title="Yaw [o]"))
fig.show()

In [15]:

#show location, possibly wrong:
#to plot things on a world map, interactive, may be incorrect.
fig = px.scatter_geo(mydata,lat=mydata.Lat,lon=mydata.Long,hover_data=[mydata.newdate] )
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()


In [ ]:
#to get speed of ISS from our location and time data
import geopy.distance
distance = np.empty([len(mydata)-1,1])
speed = np.empty([len(mydata)-1,1])
duration = np.empty([len(mydata)-1,1])
for i in range(len(mydata)-1):
  distance[i]=geopy.distance.geodesic((mydata.Lat[i],mydata.Long[i]),(mydata.Lat[i+1],mydata.Long[i+1])).km
  tmp=mydata.newdate[i+1]-mydata.newdate[i]
  duration[i]=tmp.total_seconds()
  speed[i]=distance[i]/duration[i]
  #print(speed[i])

print('Mean speed:', np.mean(speed),'km/s', '  Mean duration:', np.mean(duration),'s', '  Mean distance:', np.mean(distance),'km')
print('Literature 7.66 km/s')
#Alternatives , e.g. if duration is mistaken due to file writing, our goal was 5 seconds
print('Mean speed (alt):', np.mean(distance)/5,'km')



Mean speed: 6.8986752564046805 km/s   Mean duration: 5.271334642681124 s   Mean distance: 36.3656219409018 km
Literature 7.66 km/s
Mean speed: 7.273124388180359 km


**Step 3:** Let's list the hypotheses.

*Hypotheses*

* H1: The sensors on AstroPi - inertial measurement unit, in particular  accelerometer, gyroscope - can be utilized to separate different directions of acceleration and identify the strength of the earth’s gravitational pull on the ISS.
* H2: The distance from the earth’s core is the main determinant for the strength of gravity which we have calculated to be 8.7 cm2/sec. It will not depend on the path of the ISS since the height of the ISS is approximately constant throughout. 
* H3: Similar considerations are true for the magnetic field except that if we end up recording over the South Atlantic anomaly, we will see a strong change in the magnetic field.
